In [ ]:
"""
debug_alignment.py

This script helps debug coordinate mismatches between:
1) An original "_labeled" .ply + .npz file (with red=0, green=1).
2) A Meshlab-edited "_plant_only_fixed.ply" file, where the user manually
   deleted incorrect green points.

Steps:
  1) Load the original labeled .ply and .npz, check bounding boxes and random points.
  2) Load the Meshlab-edited .ply (plant-only).
  3) Compare bounding boxes, random points, and see if there's a large scale or shift.
  4) Build a cKDTree to compute distances from originally-labeled plant points to
     the fixed cloud, print min/mean/max distances.
  5) If needed, attempt an alignment (ICP) if there's a small mismatch. For large
     scaling differences, you'll need to fix the transform in Meshlab or apply the same
     transform to the original cloud.
"""
import os
import sys
import numpy as np
import open3d as o3d
import random
from scipy.spatial import cKDTree

# 1) Figure out paths
notebook_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
project_dir 

In [ ]:
# -----------------------------------------------------------------------------
# USER PARAMETERS (Adjust these paths to match the data)
# -----------------------------------------------------------------------------

# 1) Figure out paths
notebook_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# 2) Build file paths
ORIGINAL_LABELED_PLY = os.path.join(
    project_dir, 
    "data", 
    "manually_adjustments", 
    "Wheat_Alsen_F0_2023-06-30-1949_fused_output_labeled.ply"
)
FIXED_PLY = os.path.join(
    project_dir, 
    "data", 
    "manually_adjustments", 
    "Wheat_Alsen_F0_2023-06-30-1949_fused_output_labeled_plant_only_fixed.ply"
)
LABELED_NPZ = os.path.join(
    project_dir, 
    "data",
    "manually_adjustments",
    "Wheat_Alsen_F0_2023-06-30-1949_fused_output_labeled.npz"
)

MATCH_THRESHOLD = 1e-5
DO_ICP          = False

print("[INFO] Using paths:")
print("  ORIGINAL_LABELED_PLY =", ORIGINAL_LABELED_PLY)
print("  FIXED_PLY            =", FIXED_PLY)
print("  LABELED_NPZ          =", LABELED_NPZ)


In [ ]:
#-----------------------------------------------------------------------------
# 1) Load Original Labeled Cloud
# -----------------------------------------------------------------------------
print("=== Step 1: Load Original Labeled Cloud ===")
pcd_combined = o3d.io.read_point_cloud(ORIGINAL_LABELED_PLY)
combined_points = np.asarray(pcd_combined.points)
print(f"[INFO] Original Labeled PLY => {len(combined_points)} points.")
o3d.visualization.draw_geometries([pcd_combined])
# Load the .npz which contains 'labels' (and possibly 'points')
data = np.load(LABELED_NPZ)
labels = data["labels"]
print(f"[INFO] .npz => labels shape: {labels.shape}")

if len(combined_points) != len(labels):
    raise ValueError(f"Mismatch: {len(combined_points)} points vs. {len(labels)} labels.")

# Check bounding box
min_bound = combined_points.min(axis=0)
max_bound = combined_points.max(axis=0)
print("Bounding box of original labeled cloud:")
print("  min:", min_bound)
print("  max:", max_bound)

print("\nRandom sample points from the original cloud (with labels):")
for _ in range(5):
    idx = random.randint(0, len(combined_points)-1)
    print(f"  Index={idx}, point={combined_points[idx]}, label={labels[idx]}")


In [ ]:
# -----------------------------------------------------------------------------
# 2) Load the Meshlab-Edited Plant-Only Cloud
# -----------------------------------------------------------------------------
print("\n=== Step 2: Load Meshlab-Edited Plant-Only Cloud ===")
pcd_fixed = o3d.io.read_point_cloud(FIXED_PLY)
fixed_points = np.asarray(pcd_fixed.points)
print(f"[INFO] Meshlab-Fixed PLY => {len(fixed_points)} points.")

if len(fixed_points) > 0:
    min_bound_fix = fixed_points.min(axis=0)
    max_bound_fix = fixed_points.max(axis=0)
    print("Bounding box of Meshlab-Fixed (Plant-Only) Cloud:")
    print("  min:", min_bound_fix)
    print("  max:", max_bound_fix)

    print("\nRandom sample points from the fixed cloud:")
    for _ in range(min(5, len(fixed_points))):
        idx = random.randint(0, len(fixed_points)-1)
        print(f"  Index={idx}, point={fixed_points[idx]}")
else:
    print("[WARNING] The Meshlab-edited cloud is empty! If you proceed, all plant-labeled points would be turned to 0.")


In [ ]:
# -----------------------------------------------------------------------------
# 3) Compare bounding boxes quickly
# -----------------------------------------------------------------------------
if len(fixed_points) > 0:
    dist_bb = np.linalg.norm((min_bound - min_bound_fix)) + np.linalg.norm((max_bound - max_bound_fix))
    print(f"\n[INFO] sum of bounding box diffs => {dist_bb:.3f}")
    print("If this is huge, it suggests a major translation or scale difference.")

In [7]:
def dictionary_based_labels_update(
    combined_points: np.ndarray,
    labels: np.ndarray,
    fixed_points: np.ndarray,
) -> np.ndarray:
    """
    Approach:
      - We assume EXACT coordinate matches (no scaling or offset).
      - Convert each point in 'fixed_points' to a set/dict for O(1) lookups.
      - For each originally-labeled green point in 'combined_points', check
        if it's still in 'fixed_points'. If not => set label=0.

    Returns:
        final_labels (np.ndarray): Updated array of size len(labels).
    """
    print("[INFO] Building dictionary for 'fixed_points' ...")
    # Round or keep as-is if we trust exact float matches
    # e.g. we can do tuple(map(float, fixed_points[i]))
    # but be careful with floating precision

    # For safety, let's do a small rounding or integer transform if needed
    # e.g. round to 5 decimals:
    fixed_set = set()
    for pt in fixed_points:
        # Here we'll keep 6 decimals, adapt as needed
        key = (round(pt[0], 6), round(pt[1], 6), round(pt[2], 6))
        fixed_set.add(key)

    final_labels = labels.copy()
    plant_indices = np.where(labels == 1)[0]
    plant_points_original = combined_points[plant_indices]

    removed_count = 0
    for idx, pindex in enumerate(plant_indices):
        pt = plant_points_original[idx]
        key = (round(pt[0], 6), round(pt[1], 6), round(pt[2], 6))
        if key not in fixed_set:
            # Mark as removed
            final_labels[pindex] = 0
            removed_count += 1

    print(f"[INFO] Dictionary-based approach => {removed_count} of {len(plant_indices)} plant-labeled points turned to 0")
    return final_labels

def save_colored_ply(points: np.ndarray, labels: np.ndarray, ply_path: str):
    """
    Save a color-coded .ply for visual checking:
      green = label=1
      red   = label=0
    """
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    final_colors = np.zeros((len(points), 3))
    final_colors[labels == 1] = [0, 1, 0]
    final_colors[labels == 0] = [1, 0, 0]
    pcd.colors = o3d.utility.Vector3dVector(final_colors)
    o3d.io.write_point_cloud(ply_path, pcd, write_ascii=True)

In [ ]:
# -----------------------------------------------------------------------------
# 5) (Optional) Attempt a quick ICP alignment
# -----------------------------------------------------------------------------
if DO_ICP and len(fixed_points) > 0:
    print("\n=== Step 4: Attempting ICP to see if alignment helps ===")
    # We'll do a naive alignment: treat the Meshlab plant-only as 'source'
    # and the entire original cloud as 'target' (or we can just do the plant subset).
    threshold_icp = 1.0
    trans_init = np.eye(4)

    # We need to convert to an open3d geometry if we want to do ICP:
    source_pcd = pcd_fixed   # the partial plant-only
    target_pcd = pcd_combined

    icp_result = o3d.pipelines.registration.registration_icp(
        source_pcd,
        target_pcd,
        threshold_icp,
        trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )

    print("ICP fitness:", icp_result.fitness, " RMS:", icp_result.inlier_rmse)
    print("ICP transformation:\n", icp_result.transformation)

print("\n=== Done! ===")

In [10]:
def color_code_points(points, labels):
    """
    Create an open3d PointCloud with red=0, green=1 coloring.
    Returns o3d.geometry.PointCloud for optional visualization.
    """
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    final_colors = np.zeros((len(points), 3))
    final_colors[labels == 1] = [0,1,0]
    final_colors[labels == 0] = [1,0,0]
    pcd.colors = o3d.utility.Vector3dVector(final_colors)
    return pcd

def visualize_in_open3d(pcd):
    """
    Launch an interactive Open3D window to view the point cloud.
    Press 'q' to exit the window.
    """
    o3d.visualization.draw_geometries([pcd])

In [ ]:
def dictionary_based_labels_update(
    combined_points: np.ndarray,
    labels: np.ndarray,
    fixed_points: np.ndarray,
    decimals: int = 6
) -> np.ndarray:
    """
    For exact coordinate matching:
      - We assume no transform in Meshlab => identical coords.
      - Round coords to 'decimals' places to reduce float noise.
      - Build a set of all fixed_points coords -> check if original green coords remain.
    """
    print("\n=== Dictionary/Hash matching approach ===")
    final_labels = labels.copy()
    plant_indices = np.where(labels == 1)[0]
    plant_points_original = combined_points[plant_indices]
    print(f"# of plant-labeled => {len(plant_indices)}")

    # Build a set of all 'fixed_points'
    fixed_set = set()
    for pt in fixed_points:
        key = (round(pt[0], decimals), round(pt[1], decimals), round(pt[2], decimals))
        fixed_set.add(key)

    removed_count = 0
    for idx, pindex in enumerate(plant_indices):
        pt = plant_points_original[idx]
        key = (round(pt[0], decimals), round(pt[1], decimals), round(pt[2], decimals))
        if key not in fixed_set:
            final_labels[pindex] = 0
            removed_count += 1

    print(f"[INFO] Dictionary => {removed_count} of {len(plant_indices)} green points turned to 0.\n")
    return final_labels

USE_DICTIONARY       = True # Attempt an exact-coord dictionary-based approach
DO_VISUALIZE         = True  # Show final colored cloud(s) in Open3D interactive window
# Step 5: Dictionary-based approach (if exact coords are the same)
final_labels_dict = None
final_pcd_dict = None
if USE_DICTIONARY and len(fixed_points) > 0:
    final_labels_dict = dictionary_based_labels_update(combined_points, labels, fixed_points)
    final_pcd_dict = color_code_points(combined_points, final_labels_dict)

# Step 6: Visualization
# We can visualize the cKDTree result, the dictionary approach, or both
if DO_VISUALIZE:
    print("\n[INFO] Launching visualization window(s)...\n")
    # We'll store clouds in a list for separate windows or just one window
    # For clarity, let's open them separately.

    if final_pcd_dict is not None:
        print("=> Dictionary-based Result")
        visualize_in_open3d(final_pcd_dict)

    print("\n=== Done. ===")

